In [1]:
import os
import numpy as np
import pandas as pd
import time
import re
from typing import List, Optional
from sqlalchemy import create_engine, Column, String
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Integer, String, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base, Mapped, mapped_column, DeclarativeBase, Session
from scrapy.utils.project import get_project_settings


In [2]:

class DuplicateItem(Exception):
    pass


## Writers and Loaders

In [3]:
class DataAppender(object):
    def __init__(self):
        pass


In [65]:

class DataWriter(object):
    def __init__(self):
        pass


class DataLoader(object):
    def __init__(self):
        pass


class JsonLineWriter(DataWriter):
    def __init__(self):
        pass


class CsvReader(DataLoader):
    def __init__(self, file_path, file_name):
        self.file_path = file_path
        # self.file_name = file_name
        # Ensure the file name ends with '.csv'
        self.file_name = self.parse_input_file_name(file_name)
        # self.file_name = file_name

    def parse_input_file_name(self, file_name):
        if file_name is None:
            return None
        return re.sub(r'\.csv$', '', file_name)

    def load_data(self):
        data = pd.DataFrame()
        full_path = os.path.join(self.file_path, self.file_name)
        if not full_path.endswith('.csv'):
            full_path += '.csv'
        if os.path.exists(full_path):
            data = pd.read_csv(full_path, index_col=0)

        return data


class CsvWriter(DataWriter):
    def __init__(self, file_path, file_name=None, add_timestamp=False):
        self.file_path = file_path
        self.file_name = self.parse_input_file_name(file_name)
        self.add_timestamp = add_timestamp

    def parse_input_file_name(self, file_name):
        if file_name is None:
            return None
        return re.sub(r'\.csv$', '', file_name)

    def _write(self, data: pd.DataFrame):
        file_name = self._create_filename()
        data.to_csv(os.path.join(self.file_path, file_name))

    def _create_filename(self):
        base_name = self.file_name if self.file_name is not None else 'data'
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        if self.add_timestamp:
            file_name = f"{base_name}_{timestamp}.csv"
        else:
            file_name = f"{base_name}.csv"
        return file_name

    def _convert_to_dataframe(self, items) -> pd.DataFrame:
        if isinstance(items, dict):
            return pd.DataFrame.from_dict(data=items, orient='columns')
        if isinstance(items, pd.Series):
            return pd.DataFrame(items)
        if isinstance(items, pd.DataFrame):
            return pd.concat([items])
        if not items:
            return pd.DataFrame()

    def write(self, items):
        new_entries = self._convert_to_dataframe(items)
        if new_entries.empty:
            return
        self._write(new_entries)


class CsvAppender(CsvWriter):
    def __init__(self, reader, avoid_duplicates=False, duplicate_columns=[]):
        self.reader = reader
        self.avoid_duplicates = avoid_duplicates
        self.duplicate_columns = duplicate_columns

        self.file_path = reader.file_path
        self.file_name = reader.file_name
        self.add_timestamp = False

    def load_data(self):
        if self.reader is not None:
            data = self.reader.load_data()
            # print(f"Loading data. Shape: {data.shape}")

        else:
            data = pd.DataFrame()
        return data

    def filter_duplicates(self, data, new_entries):
        if data.empty:
            return new_entries
        columns = self.duplicate_columns
        if self.duplicate_columns == []:
            columns = data.columns
        bool_mask = np.any([~new_entries[col].isin(data[col]) for col in columns], axis=0)
        new_entries = new_entries[bool_mask]
        return new_entries

    def write(self, items: list):
        new_entries = self._convert_to_dataframe(items)
        if new_entries.empty:
            return

        data = self.load_data()

        if self.avoid_duplicates:
            unique_entries = self.filter_duplicates(data, new_entries)
            if unique_entries.empty:
                return
            entries = pd.concat([data, unique_entries])
        else:
            entries = pd.concat([data, new_entries])
            print(f"{entries.shape} {data.shape} {new_entries.shape}")

        self._write(entries)




In [5]:

class Base(DeclarativeBase):
    pass


class SqliteWriter(DataWriter):

    def __init__(self,
        # db_path: str,# table_name: str,# columns: List[str],
        avoid_duplicates: bool,
        duplicate_columns: List[str] = [], #None
    ):
        # self.db_path = db_path# self.table_name = table_name# self.columns = columns
        self.avoid_duplicates = avoid_duplicates
        self.duplicate_columns = duplicate_columns or []

        # self.conn = sqlite3.connect(self.db_path)# self.cursor = self.conn.cursor()# self.create_table()
        # ###############
        engine = self.db_connect()
        self.create_table(engine)
        self.factory = sessionmaker(bind=engine)

    def db_connect(self):
        """
        Performs database connection using database settings from settings.py.
        Returns sqlaclchemy engine instance.
        """
        url = get_project_settings().get("CONNECTION_STRING")
        return create_engine(url)

    def create_table(self, engine):
        Base.metadata.create_all(engine, checkfirst=True)

    def check_duplicate(self, item): #, session=None
        session = self.factory()
        # session = session if session is not None else self.factory()
        exist_title = session.query(VivaRealCatalog).filter_by(title=item["title"]).first()
        session.close()
        if (exist_title is not None):
            print("Duplicate item found: {}".format(item["title"]))
            # raise DropItem("Duplicate item found: {}".format(item["title"]))
        else:
            return item

    def write(self, item): #, session=None
        # session = self.factory()
        try:
            self.check_duplicate(item) #, session
            self.process_item(item) #, session
        except DuplicateItem as e:
            print(e)
        # finally:
        #     session.close()

    def process_item(self, item):
        """
        This method is called for every item pipeline component
        """
        session = self.factory()
        # session = session if session is not None else self.factory()
        catalog = VivaRealCatalog()
        catalog.type = item["type"]
        catalog.address = item["address"]
        catalog.title = item["title"]
        catalog.details = item["details"]
        catalog.amenities = item["amenities"]
        catalog.values = item["values"]
        catalog.target_url = item["target_url"]
        catalog.catalog_scraped_date = item['catalog_scraped_date']
        catalog.is_target_scraped = item["is_target_scraped"]

        try:
            print('Entry added')
            session.add(catalog)
            session.commit()
        except:
            print('rollback')
            session.rollback()
            raise
        finally:
            session.close()

        return item


## Database

In [6]:

class VivaRealCatalog(Base):
    __tablename__ = "vivareal_catalog"

    id: Mapped[int] = mapped_column(primary_key=True)
    type: Mapped[str] = mapped_column(String(20))
    address: Mapped[str] = mapped_column(String(200))
    title: Mapped[str] = mapped_column(String(200)) # Mapped[Optional[str]]
    details: Mapped[str] = mapped_column(String(200))
    amenities: Mapped[Optional[str]] = mapped_column(String(200))
    values: Mapped[str] = mapped_column(String(200))
    target_url: Mapped[str] = mapped_column(String(200))
    # catalog_scraped_date: Mapped[DateTime] = mapped_column(DateTime)
    catalog_scraped_date: Mapped[str] = mapped_column(String(30))
    is_target_scraped: Mapped[int] = mapped_column(Integer)


# address: "Rua Doutor Otorino Avancini, 606 - Nova Itaparica, Vila Velha - ES"
# amenities: Portão eletrônico Área de serviço Armário na cozinha Armário no banheiro Box blindex
# catalog_scraped_date: 2024-09-03T12:37:09.792558
# details: 55 m²<br>2 Quartos<br>1 Banheiro<br>-- Vaga
# is_target_scraped: 0
# target_url: https://www.vivareal.com.br/imovel/apartamento-2-quartos-nova-itaparica-bairros-vila-velha-55m2-aluguel-RS1300-id-2738160163/
# title: "Apartamento com 2 Quartos para Aluguel, 55m²"
# type: catalog
# values: R$ 1.300 /Mês


## Experiments

In [7]:
def str2df(input_str):
    # Split the input string into lines
    lines = input_str.strip().split('\n')

    # Extract the header and the data rows
    header = lines[0].split(',')
    data_rows = lines[1:]

    # Parse the data rows into a list of dictionaries
    data = []
    for row in data_rows:
        # Use regex to split the row by commas, but ignore commas inside quotes
        row_data = re.split(r',(?=(?:[^"]*"[^"]*")*[^"]*$)', row)
        # Remove quotes from each field
        row_data = [field.strip('"') for field in row_data]
        # Create a dictionary for the row
        row_dict = dict(zip(header, row_data))
        data.append(row_dict)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(data)

    # Set the first column of the dataframe as the index
    df.set_index(df.columns[0], inplace=True)

    return df


input_str1 = """
,address,amenities,catalog_scraped_date,details,is_target_scraped,target_url,title,type,values
0,"Rua Doutor Otorino Avancini, 606 - Nova Itaparica, Vila Velha - ES","Portão eletrônico Área de serviço Armário na cozinha Armário no banheiro Box blindex",2024-09-03T12:37:09.792558,"55 m²<br>2 Quartos<br>1 Banheiro<br>-- Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-nova-itaparica-bairros-vila-velha-55m2-aluguel-RS1300-id-2738160163/,"Apartamento com 2 Quartos para Aluguel, 55m²",catalog,"R$ 1.300 /Mês"
1,"Rua Vinícius de Morais - Nova Itaparica, Vila Velha - ES","none",2024-09-03T12:37:09.802709,"430 m²<br>-- Quarto<br>-- Banheiro<br>-- Vaga",0,https://www.vivareal.com.br/imovel/galpao-deposito-armazem-nova-itaparica-bairros-vila-velha-430m2-aluguel-RS7500-id-2715660696/,"Galpão/Depósito/Armazém para Aluguel, 430m²",catalog,"R$ 7.500 /Mês"
0,"Rua Coronel Otto Netto - Jockey de Itaparica, Vila Velha - ES","Piscina<br>Churrasqueira<br>Elevador<br>Varanda<br>Academia<br>...",2024-09-03T12:37:23.169182,"54 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-54m2-aluguel-RS2000-id-2734570841/,"Apartamento com 2 Quartos para Aluguel, 54m²",catalog,"R$ 2.000 /Mês<br>Condomínio: R$ 384"
1,"Avenida dos Estados, 10 - Jockey de Itaparica, Vila Velha - ES","Portão eletrônico<br>Elevador<br>Bicicletário",2024-09-03T12:37:23.181741,"68 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-68m2-aluguel-RS1550-id-2737910909/,"Apartamento com 2 Quartos para Aluguel, 68m²",catalog,"R$ 1.550 /Mês<br>Preço abaixo do mercado<br>Condomínio: R$ 250"
2,"Rua Porto Seguro, 286 - Jockey de Itaparica, Vila Velha - ES","Lavanderia<br>Portão eletrônico",2024-09-03T12:37:23.203799,"35 m²<br>1 Quarto<br>1 Banheiro<br>-- Vaga",0,https://www.vivareal.com.br/imovel/kitnet-1-quartos-jockey-de-itaparica-bairros-vila-velha-35m2-aluguel-RS950-id-2706987955/,"Apartamento com Quarto para Aluguel, 35m²",catalog,"R$ 950 /Mês"
3,"Avenida Ceará, 500 - Jockey de Itaparica, Vila Velha - ES","Mobiliado<br>Churrasqueira<br>Condomínio fechado<br>Aceita animais<br>Playground<br>...",2024-09-03T12:37:23.220110,"51 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-51m2-aluguel-RS1700-id-2733888516/,"Apartamento com 2 Quartos para Aluguel, 51m²",catalog,"R$ 1.700 /Mês<br>Condomínio: R$ 147"
4,"Avenida Ceará, 300 - Jockey de Itaparica, Vila Velha - ES","Condomínio fechado",2024-09-03T12:37:23.234845,"50 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-50m2-aluguel-RS1100-id-2737985914/,"Apartamento com 2 Quartos para Aluguel, 50m²",catalog,"R$ 1.100 /Mês<br>Condomínio: R$ 300"
5,"Avenida Ceará - Jockey de Itaparica, Vila Velha - ES","Condomínio fechado",2024-09-03T12:37:23.242894,"52 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-52m2-aluguel-RS1700-id-2736845623/,"Apartamento com 2 Quartos para Aluguel, 52m²",catalog,"R$ 1.700 /Mês<br>Condomínio: R$ 290"
6,"Rua Coronel Otto Netto - Jockey de Itaparica, Vila Velha - ES","Piscina<br>Churrasqueira<br>Elevador<br>Academia<br>Aceita animais<br>...",2024-09-03T12:37:23.263289,"60 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-60m2-aluguel-RS2300-id-2733818315/,"Apartamento com 2 Quartos para Aluguel, 60m²",catalog,"R$ 2.300 /Mês<br>Condomínio: R$ 350"
"""

input_str2 = """
,address,amenities,catalog_scraped_date,details,is_target_scraped,target_url,title,type,values
0,"Rua Doutor Otorino Avancini, 606 - Nova Itaparica, Vila Velha - ES","Portão eletrônico Área de serviço Armário na cozinha Armário no banheiro Box blindex",2024-09-03T12:37:09.792558,"55 m²<br>2 Quartos<br>1 Banheiro<br>-- Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-nova-itaparica-bairros-vila-velha-55m2-aluguel-RS1300-id-2738160163/,"Apartamento com 2 Quartos para Aluguel, 55m²",catalog,"R$ 1.300 /Mês"
1,"Rua Vinícius de Morais - Nova Itaparica, Vila Velha - ES","none",2024-09-03T12:37:09.802709,"430 m²<br>-- Quarto<br>-- Banheiro<br>-- Vaga",0,https://www.vivareal.com.br/imovel/galpao-deposito-armazem-nova-itaparica-bairros-vila-velha-430m2-aluguel-RS7500-id-2715660696/,"Galpão/Depósito/Armazém para Aluguel, 430m²",catalog,"R$ 7.500 /Mês"
"""

input_str3 = """
,address,amenities,catalog_scraped_date,details,is_target_scraped,target_url,title,type,values
0,"Rua Coronel Otto Netto - Jockey de Itaparica, Vila Velha - ES","Piscina<br>Churrasqueira<br>Elevador<br>Varanda<br>Academia<br>...",2024-09-03T12:37:23.169182,"54 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-54m2-aluguel-RS2000-id-2734570841/,"Apartamento com 2 Quartos para Aluguel, 54m²",catalog,"R$ 2.000 /Mês<br>Condomínio: R$ 384"
1,"Avenida dos Estados, 10 - Jockey de Itaparica, Vila Velha - ES","Portão eletrônico<br>Elevador<br>Bicicletário",2024-09-03T12:37:23.181741,"68 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-68m2-aluguel-RS1550-id-2737910909/,"Apartamento com 2 Quartos para Aluguel, 68m²",catalog,"R$ 1.550 /Mês<br>Preço abaixo do mercado<br>Condomínio: R$ 250"
2,"Rua Porto Seguro, 286 - Jockey de Itaparica, Vila Velha - ES","Lavanderia<br>Portão eletrônico",2024-09-03T12:37:23.203799,"35 m²<br>1 Quarto<br>1 Banheiro<br>-- Vaga",0,https://www.vivareal.com.br/imovel/kitnet-1-quartos-jockey-de-itaparica-bairros-vila-velha-35m2-aluguel-RS950-id-2706987955/,"Apartamento com Quarto para Aluguel, 35m²",catalog,"R$ 950 /Mês"
3,"Avenida Ceará, 500 - Jockey de Itaparica, Vila Velha - ES","Mobiliado<br>Churrasqueira<br>Condomínio fechado<br>Aceita animais<br>Playground<br>...",2024-09-03T12:37:23.220110,"51 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-51m2-aluguel-RS1700-id-2733888516/,"Apartamento com 2 Quartos para Aluguel, 51m²",catalog,"R$ 1.700 /Mês<br>Condomínio: R$ 147"
4,"Avenida Ceará, 300 - Jockey de Itaparica, Vila Velha - ES","Condomínio fechado",2024-09-03T12:37:23.234845,"50 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-50m2-aluguel-RS1100-id-2737985914/,"Apartamento com 2 Quartos para Aluguel, 50m²",catalog,"R$ 1.100 /Mês<br>Condomínio: R$ 300"
5,"Avenida Ceará - Jockey de Itaparica, Vila Velha - ES","Condomínio fechado",2024-09-03T12:37:23.242894,"52 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-52m2-aluguel-RS1700-id-2736845623/,"Apartamento com 2 Quartos para Aluguel, 52m²",catalog,"R$ 1.700 /Mês<br>Condomínio: R$ 290"
6,"Rua Coronel Otto Netto - Jockey de Itaparica, Vila Velha - ES","Piscina<br>Churrasqueira<br>Elevador<br>Academia<br>Aceita animais<br>...",2024-09-03T12:37:23.263289,"60 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga",0,https://www.vivareal.com.br/imovel/apartamento-2-quartos-jockey-de-itaparica-bairros-vila-velha-com-garagem-60m2-aluguel-RS2300-id-2733818315/,"Apartamento com 2 Quartos para Aluguel, 60m²",catalog,"R$ 2.300 /Mês<br>Condomínio: R$ 350"
"""

curr_path = os.getcwd()
base_path = os.path.abspath(os.path.join(curr_path, os.pardir))
file_path = os.path.join(base_path, 'data')
file_name = 'foo.csv'

# some_df = str2df(input_str1)
# print(some_df.columns)
# some_df

In [16]:


# str2df(input_str1)


# some_item = {
#     'address': 'Rua Doutor Otorino Avancini, 606 - Nova Itaparica, Vila Velha - ES',
#     'amenities': 'Portão eletrônico Área de serviço Armário na cozinha Armário no banheiro Box blindex',
#     'catalog_scraped_date': '2024-09-03T22:46:31.381278',
#     'details': '55 m²<br>2 Quartos<br>1 Banheiro<br>-- Vaga',
#     'is_target_scraped': 0,
#     'target_url': 'https://www.vivareal.com.br/imovel/apartamento-2-quartos-nova-itaparica-bairros-vila-velha-55m2-aluguel-RS1300-id-2738160163/',
#     'title': 'Apartamento com 2 Quartos para Aluguel, 55m²',
#     'type': 'catalog',
#     'values': 'R$ 1.300 /Mês'
# }

# sql_writer = SqliteWriter(avoid_duplicates=True)
# sql_writer.write(some_item)

# print("EOL")


### 1. Data lake style writing
Description: Create different CSV files for each scraping session

In [69]:

writer = CsvWriter(file_path=file_path, file_name='custom_name', add_timestamp=True)
writer.write(str2df(input_str2))

writer = CsvWriter(file_path=file_path, file_name='with_duplicates', add_timestamp=False)
writer.write(str2df(input_str2))

writer = CsvWriter(file_path=file_path, file_name='without_duplicates', add_timestamp=False)
writer.write(str2df(input_str2))

writer = CsvWriter(file_path=file_path, add_timestamp=True)
writer.write(str2df(input_str2))

writer = CsvWriter(file_path=file_path)
writer.write(str2df(input_str2))


### 1.5 Reading data

In [68]:
reader = CsvReader(file_path=file_path, file_name='with_duplicates')
data = reader.load_data()
print(f"Shape of data: {data.shape}")
data.head(1)

Shape of data: (2, 9)


,address,amenities,catalog_scraped_date,details,is_target_scraped,target_url,title,type,values
0,"Rua Doutor Otorino Avancini, 606 - Nova Itapar...",Portão eletrônico Área de serviço Armário na c...,2024-09-03T12:37:09.792558,55 m²<br>2 Quartos<br>1 Banheiro<br>-- Vaga,0,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 55m²",catalog,R$ 1.300 /Mês


### 2. Data warehouse style writing
Description: Append the scraped data to a single CSV file, including or excluding duplicates

In [74]:
# Test case: save input_str3 to with_duplicates 2 times
writer = CsvAppender(
    reader=CsvReader(file_path=file_path, file_name="with_duplicates.csv"),
    avoid_duplicates=False,
    duplicate_columns=['address', 'title', 'values'],
    # write_in_batches=True,
)
writer.write(str2df(input_str3))
writer.write(str2df(input_str3))


# Test case: save input_str3 to without_duplicates 2 times
writer = CsvAppender(
    reader=CsvReader(file_path=file_path, file_name="without_duplicates.csv"),
    avoid_duplicates=True,
    duplicate_columns=['address', 'title', 'values'],
    # write_in_batches=True,
)
writer.write(str2df(input_str3))
writer.write(str2df(input_str3))


(37, 9) (30, 9) (7, 9)
(44, 9) (37, 9) (7, 9)


In [75]:
foo = pd.read_csv(os.path.join(file_path, 'with_duplicates.csv'), index_col=0)
bar = pd.read_csv(os.path.join(file_path, 'without_duplicates.csv'), index_col=0)

print(f"Shape of foo: {foo.shape}")
print(f"Shape of bar: {bar.shape}")


Shape of foo: (44, 9)
Shape of bar: (9, 9)


In [18]:
bar

,address,amenities,catalog_scraped_date,details,is_target_scraped,target_url,title,type,values
0,"Rua Coronel Otto Netto - Jockey de Itaparica, ...",Piscina<br>Churrasqueira<br>Elevador<br>Varand...,2024-09-03T12:37:23.169182,54 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga,0,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 54m²",catalog,R$ 2.000 /Mês<br>Condomínio: R$ 384
1,"Avenida dos Estados, 10 - Jockey de Itaparica,...",Portão eletrônico<br>Elevador<br>Bicicletário,2024-09-03T12:37:23.181741,68 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga,0,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 68m²",catalog,R$ 1.550 /Mês<br>Preço abaixo do mercado<br>Co...
2,"Rua Porto Seguro, 286 - Jockey de Itaparica, V...",Lavanderia<br>Portão eletrônico,2024-09-03T12:37:23.203799,35 m²<br>1 Quarto<br>1 Banheiro<br>-- Vaga,0,https://www.vivareal.com.br/imovel/kitnet-1-qu...,"Apartamento com Quarto para Aluguel, 35m²",catalog,R$ 950 /Mês
3,"Avenida Ceará, 500 - Jockey de Itaparica, Vila...",Mobiliado<br>Churrasqueira<br>Condomínio fecha...,2024-09-03T12:37:23.220110,51 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga,0,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 51m²",catalog,R$ 1.700 /Mês<br>Condomínio: R$ 147
4,"Avenida Ceará, 300 - Jockey de Itaparica, Vila...",Condomínio fechado,2024-09-03T12:37:23.234845,50 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga,0,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 50m²",catalog,R$ 1.100 /Mês<br>Condomínio: R$ 300
5,"Avenida Ceará - Jockey de Itaparica, Vila Velh...",Condomínio fechado,2024-09-03T12:37:23.242894,52 m²<br>2 Quartos<br>1 Banheiro<br>1 Vaga,0,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 52m²",catalog,R$ 1.700 /Mês<br>Condomínio: R$ 290
6,"Rua Coronel Otto Netto - Jockey de Itaparica, ...",Piscina<br>Churrasqueira<br>Elevador<br>Academ...,2024-09-03T12:37:23.263289,60 m²<br>2 Quartos<br>2 Banheiros<br>1 Vaga,0,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 60m²",catalog,R$ 2.300 /Mês<br>Condomínio: R$ 350


### 3. Database style writing
Description: Write the scraped data to a SQLite database, including or excluding duplicates

In [49]:

writer = SqliteWriter(
    avoid_duplicates=True,
    duplicate_columns=['address', 'title', 'values'],
)

KeyboardInterrupt: 